### Input: ROOT files with the THnSparses of the same-event and mixed-event correlations, ROOT file with response matrices, and the configuration files used to generate those correlations and response matrices

This notebook is just for plotting purposes. It uses the GammaJetCorrelation class, which converts the various histograms to numpy arrays, does various subtractions, and saves the histograms as numpy arrays. The notebook then takes these arrays and generates and saves the plots.

This is designed for a single iteration (i.e. a single configuration file), so comparing different iterations together will require modification.

### Workflow:
1. Read config files
2. Pull histograms and response matrices from ROOT file(s) and save as ROOT objects
3. Make gamma-jet correlation objects, which consist of the following steps:
    1. Subtract mixed-event from same-event for both signal-region and background-region. Save these histograms as numpy arrays (for plotting)
    2. Subtract background-region from signal-region for both same-event and mixed-event. Save these histograms as numpy arrays (for plotting)
    3. Subtract background-region from signal-region. Save this histogram as a numpy array (for plotting)
    4. Convert this fully-subtracted correlation to a TH1 for unfolding
    5. Unfold with response matrix
    6. Convert unfolded correlation to numpy array (for plotting)
4. Make and save plots

In [ ]:
workingdir=%pwd
%cd -q ../..

from __future__ import print_function

import matplotlib.pyplot as plt
import numpy as np
import os
import ROOT
import root_numpy as rnp
import shutil
import yaml

from gj_correlation import GammaJetCorrelation, getAllCorr
from plotstyle import *
from unfolder import Unfolder
from utils import *

ROOT.TH1.AddDirectory(False)
ROOT.TH2.AddDirectory(False)

%cd $workingdir
plotdir = 'plots'
if not os.path.exists(plotdir):
    os.mkdir(plotdir)

In [ ]:
# load all YAML files
with open('config/globalconfig.yaml') as configfile:
    configgj = yaml.safe_load(configfile)
    
with open('config/systemconfig.yaml') as configfile:
    configsys = yaml.safe_load(configfile)

with open('config/runconfig.yaml') as configfile:
    configrun = yaml.safe_load(configfile)

configall = {}
configall.update(configgj)
configall.update(configsys)
configall.update(configrun)

centranges = [tuple(centrange) for centrange in configsys['centralityranges']]
photonptranges = [tuple(ptrange) for ptrange in configsys['clusterptranges']]
observables = [obs['name'] for obs in configgj['observables']]
observableInfo = {obs['name']: obs for obs in configgj['observables']}

for observable in observables:
    obsdir = '{0}/{1}'.format(plotdir, observable)
    if not os.path.exists(obsdir):
        os.mkdir(obsdir)

In [ ]:
# make any *minor* changes to the configuration here
# (e.g. jetpt cut, deltaphi cut, number of bins -- things that do not affect the creation of the THnSparses)
# this should only be done for exploring these variations
# any real changes should be made in the (global) configuration file
def setJetPtRange(ptmin, ptmax):
    observableInfo['deltaphi']['cuts']['jetpt'] = {'min': ptmin, 'max': ptmax, 'text': '{0} < $p_\mathrm{{T}}^\mathrm{{jet}}$ < {1} GeV/$c$'.format(ptmin, ptmax)}
    observableInfo['ptratio']['cuts']['jetpt'] = {'min': ptmin, 'max': ptmax, 'text': '{0} < $p_\mathrm{{T}}^\mathrm{{jet}}$ < {1} GeV/$c$'.format(ptmin, ptmax)}
    
def setDeltaPhiMin(num, den):
    # num, den: the numerator and denominator of the fraction of pi to be used as the minimum deltaphi pt cut
    observableInfo['jetpt']['cuts']['deltaphi'] = {'min': np.pi * num / den, 'max': np.pi, 'text': '{0}$\pi$/{1} < $\Delta\varphi$ < $\pi$'.format(num, den)}
    observableInfo['ptratio']['cuts']['deltaphi'] = {'min': np.pi * num / den, 'max': np.pi, 'text': '{0}$\pi$/{1} < $\Delta\varphi$ < $\pi$'.format(num, den)}

def useNbins(observable, nbins):
    if 120 % nbins == 0:
        observableInfo[observable]['nbins'] = nbins
    else:
        print('Cannot use {0} bins as it is not a multiple of 120'.format(nbins))

In [ ]:
# get correlation objects
rootfileSE = configrun['filelists']['correlations']['sameevent']
rootfileME = configrun['filelists']['correlations']['mixedevent']
rootfileMEskim5090 = rootfileME.replace('.root', '_skimcent5090.root')
rootfileRM = configrun['filelists']['correlations']['responsematrix']

allCorr = getAllCorr(centranges, photonptranges, observableInfo, rootfileSE, rootfileME, rootfileRM)
allCorrskim5090 = getAllCorr([(50, 90)], photonptranges, observableInfo, rootfileSE, rootfileME, rootfileRM)
allCorr[(50, 90)] = allCorrskim5090[(50, 90)]

allCorr['longname'] = configrun['longname']
allCorr['shortname'] = configrun['shortname']

## Print configuration for future reference

In [ ]:
print(configall)

## Correlation plots

In [ ]:
# plot all raw correlations on same plot
for observable in ['deltaphi', 'ptratio']:
    for centrange in centranges:
        fig = plt.figure(figsize=(len(photonptranges) * 8, 8))
        for (icol, photonptrange) in enumerate(photonptranges):
            gjCorr = allCorr[centrange][photonptrange][observable]

            plt.subplot(1, len(photonptranges), icol + 1)
            gjCorr.plotCorr('sesr', fmt='C0o', fillstyle='none', label='SESR')
            gjCorr.plotCorr('sebr', fmt='C0o-', fillstyle='none', alpha=0.4, label='SEBR')
            gjCorr.plotCorr('mesr', fmt='C1s', fillstyle='none', label='MESR')
            gjCorr.plotCorr('mebr', fmt='C1s-', fillstyle='none', alpha=0.4, label='MEBR')

            if icol == len(photonptranges) - 1:
                plt.legend(loc=observableInfo[observable]['legendLoc'])

            if observableInfo[observable]['isLog']:
                plt.yscale('log')

            plt.xlabel(observableInfo[observable]['xlabel'])

            if icol == 0:
                plt.ylabel(observableInfo[observable]['ylabel'])
                infotext = '{0}, {1}-{2}%'.format(allCorr['longname'], centrange[0], centrange[1])
                plt.annotate(infotext, **observableInfo[observable]['systemtextplacement'])

            infotext = '{0} < $p_\mathrm{{T}}^\mathrm{{cluster}}$ < {1} GeV/$c$'.format(photonptrange[0], photonptrange[1])
            for cut in observableInfo[observable]['cuts']:
                infotext = infotext + '\n' + observableInfo[observable]['cuts'][cut]['text']
            plt.annotate(infotext, **observableInfo[observable]['pttextplacement'])

        fig.savefig('{0}/{1}/sesr-sebr-mesr-mebr-{2}-{3}-{4}.png'.format(plotdir, observable, allCorr['shortname'], centrange[0], centrange[1]))

In [ ]:
# make BR-subtracted SE and ME plots
for observable in ['deltaphi', 'ptratio']:
    for centrange in centranges:
        fig = plt.figure(figsize=(len(photonptranges) * 8, 8))
        for (icol, photonptrange) in enumerate(photonptranges):
            gjCorr = allCorr[centrange][photonptrange][observable]

            plt.subplot(1, len(photonptranges), icol + 1)
            gjCorr.plotCorr('se', fmt='C0o', label='SE')
            gjCorr.plotCorr('me', fmt='C1s', label='ME')

            if icol == len(photonptranges) - 1:
                plt.legend(loc=observableInfo[observable]['legendLoc'])

            if observableInfo[observable]['isLog']:
                plt.yscale('log')

            plt.xlabel(observableInfo[observable]['xlabel'])

            if icol == 0:
                plt.ylabel(observableInfo[observable]['ylabel'])
                infotext = '{0}, {1}-{2}%'.format(allCorr['longname'], centrange[0], centrange[1])
                plt.annotate(infotext, **observableInfo[observable]['systemtextplacement'])

            infotext = '{0} < $p_\mathrm{{T}}^\mathrm{{cluster}}$ < {1} GeV/$c$'.format(photonptrange[0], photonptrange[1])
            for cut in observableInfo[observable]['cuts']:
                infotext = infotext + '\n' + observableInfo[observable]['cuts'][cut]['text']
            plt.annotate(infotext, **observableInfo[observable]['pttextplacement'])

        fig.savefig('{0}/{1}/se-me-{2}-{3}-{4}.png'.format(plotdir, observable, allCorr['shortname'], centrange[0], centrange[1]))

In [ ]:
# make ME-subtracted SR and BR plots
for observable in ['deltaphi', 'ptratio']:
    for centrange in centranges:
        fig = plt.figure(figsize=(len(photonptranges) * 8, 8))
        for (icol, photonptrange) in enumerate(photonptranges):
            gjCorr = allCorr[centrange][photonptrange][observable]

            plt.subplot(1, len(photonptranges), icol + 1)
            gjCorr.plotCorr('sr', fmt='C0o', label='SR')
            gjCorr.plotCorr('br', fmt='C1s', label='BR')

            if icol == len(photonptranges) - 1:
                plt.legend(loc=observableInfo[observable]['legendLoc'])

            if observableInfo[observable]['isLog']:
                plt.yscale('log')

            plt.xlabel(observableInfo[observable]['xlabel'])

            if icol == 0:
                plt.ylabel(observableInfo[observable]['ylabel'])
                infotext = '{0}, {1}-{2}%'.format(allCorr['longname'], centrange[0], centrange[1])
                plt.annotate(infotext, **observableInfo[observable]['systemtextplacement'])

            infotext = '{0} < $p_\mathrm{{T}}^\mathrm{{cluster}}$ < {1} GeV/$c$'.format(photonptrange[0], photonptrange[1])
            for cut in observableInfo[observable]['cuts']:
                infotext = infotext + '\n' + observableInfo[observable]['cuts'][cut]['text']
            plt.annotate(infotext, **observableInfo[observable]['pttextplacement'])

        fig.savefig('{0}/{1}/sr-br-{2}-{3}-{4}.png'.format(plotdir, observable, allCorr['shortname'], centrange[0], centrange[1]))

In [ ]:
# make fully-subtracted correlation plots
for observable in ['deltaphi', 'ptratio']:
    for centrange in centranges:
        fig = plt.figure(figsize=(len(photonptranges) * 8, 8))
        for (icol, photonptrange) in enumerate(photonptranges):
            plt.subplot(1, len(photonptranges), icol + 1)
            corr = allCorr[centrange][photonptrange][observable]
            corr.plotCorr('corr', fmt='o', label=allCorr['longname'])
                        
            if observableInfo[observable]['isLog']:
                plt.yscale('log')
                
            plt.xlabel(observableInfo[observable]['xlabel'])
            
            if icol == 0:
                plt.ylabel(observableInfo[observable]['ylabel'])
                infotext = '{0}, {1}-{2}%'.format('Pb-Pb', centrange[0], centrange[1])
                plt.annotate(infotext, **observableInfo[observable]['systemtextplacement'])
                
            infotext = '{0} < $p_\mathrm{{T}}^\mathrm{{cluster}}$ < {1} GeV/$c$'.format(photonptrange[0], photonptrange[1])
            for cut in observableInfo[observable]['cuts']:
                infotext = infotext + '\n' + observableInfo[observable]['cuts'][cut]['text']
            plt.annotate(infotext, **observableInfo[observable]['pttextplacement'])
            
        fig.savefig('{0}/{1}/corr-measured-{2}-{3}.png'.format(plotdir, observable, centrange[0], centrange[1]))